# Exercise 4.8 task:

### Loading necessary libraries and path:

In [71]:
import pandas as pd
import numpy as np
import os

In [3]:
path = r'C:\Users\steve\Documents\CF - Instacart Project\02 Data'

In [4]:
ords_prods_merge = pd.read_pickle(os.path.join(path,'Prepared Data', 'orders_products_updated.pkl'))

In [6]:
ords_prods_merge.head(3)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most orders


### Step 2:

Find the aggregate mean of the order_number column, grouped by department_id:

In [7]:
# Finding the mean number of orders for each department
ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### Step 3:

Analyze the results, how does the entire dataframe differ from the subset from the exercise?

The entire dataframe contains every department_id, from 1-21, the subset of the first million rows only contained 8 department_ids.

Additionally, due to there being more values to account for, the mean values changed when compared between the two.

### Step 4:

Create the loyalty_flag column using transform() and loc():

In [9]:
# Getting the max order per user_id, so that we can use that as a flag value later
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)

In [20]:
ords_prods_merge.head(3)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Average orders,10,New customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average orders,10,New customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most orders,10,New customer


In [16]:
# Creating loyalty_flag column and assigning max_orders over 4o to Loyal customers
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [17]:
# Same as above but assigning max orders between 40 and 6 to Regular customers
ords_prods_merge.loc[(ords_prods_merge['max_order']<=40) & (ords_prods_merge['max_order']>10), 'loyalty_flag'] = 'Regular customer'

In [18]:
# Assigning customers with 5 or less orders to New customers
ords_prods_merge.loc[ords_prods_merge['max_order']<=10, 'loyalty_flag'] = 'New customer'

In [21]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

### Step 5:

The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [23]:
# Grouping by loyalty flag, checking the aggregates of prices for products 
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['mean', 'min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

Something is very very off about that max, nothing should cost that much.  Going to look into how to locate the item with that price and possibly remove or adjust the price if its suppose to be 9.99.

In [32]:
# Checking the counts of purchases within different ranges, grouped by loyalty
ords_prods_merge.groupby('loyalty_flag')['price_range'].value_counts()

loyalty_flag      price_range       
Loyal customer    Mid-range product      6949467
                  Low-range product      3214690
                  High-range product      119936
New customer      Mid-range product      4205569
                  Low-range product      1952769
                  High-range product       85652
Regular customer  Mid-range product     10705824
                  Low-range product      4958862
                  High-range product      212090
Name: count, dtype: int64

It looks like new customers often end up spending more money per item, most likely due to less experience shopping through the app or possibly not using deals?

Looking at the breakdown for purchases based on the price range - all three consumers prefer mid-range products, followed by low- and finally high-ranged products.

### Step 6:

Find the mean for the prices of products purchased, grouped by user id.  Then label them as a "Low spender" if their average is < 10 and a "High spender" if their average is 10 or greater.

In [60]:
# Getting a preview of the content
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.37,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.37,Low spender
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most orders,10,New customer,6.37,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.37,Low spender
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most orders,10,New customer,6.37,Low spender


In [53]:
# Creating a average_price column, grouping by user id and then getting the mean of prices
ords_prods_merge['average_price'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)

In [56]:
# Rounding the results
ords_prods_merge['average_price'] = ords_prods_merge['average_price'].round(2)

In [58]:
# Going to find all users with a average_price less than 10 and label low spender
ords_prods_merge.loc[ords_prods_merge['average_price'] < 10, 'spending_flag'] = 'Low spender'

In [59]:
# Doing the same but for >= 10 and labelling High spender
ords_prods_merge.loc[ords_prods_merge['average_price'] >= 10, 'spending_flag'] = 'High spender'

In [63]:
ords_prods_merge['spending_flag'].value_counts()

spending_flag
Low spender     31769965
High spender      634894
Name: count, dtype: int64

Number makes sense but one issue I have is that, this is just an average of the price per item they purchased.  Super ideal would be to group by user_id and then take the results of that and group by order_number as well, so we can get an average of how much they spend PER order.  Going to try some tests for that after finishing this task, so to not mess up the current chart. 

### Step 7:

Create a new column to find the median for days_since_last_order per user id.  Once that is found, flag the users based on the median with >20 being "Non-frequent customer", >10 AND <=20 "Regular customer", and <= 10 being "Frequent customer".

In [75]:
# Getting a preview of the content
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_days,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,Mid-range product,Least busy,Most orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,Mid-range product,Least busy,Most orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer


In [64]:
# Getting the median for days_since_last_order grouped by customer_id
ords_prods_merge['median_days'] = ords_prods_merge.groupby('user_id')['days_since_last_order'].transform(np.median)

In [66]:
# Creating rules and label for Non-frequent customer
ords_prods_merge.loc[ords_prods_merge['median_days'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [67]:
# Labgel for frequent customer
ords_prods_merge.loc[ords_prods_merge['median_days'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [68]:
# Label for regular customer
ords_prods_merge.loc[(ords_prods_merge['median_days'] > 10) & (ords_prods_merge['median_days'] <= 20), 'frequency_flag'] = 'Regular customer'

### Exporting File:

In [73]:
ords_prods_merge.to_pickle(os.path.join(path, 'Prepared Data', 'orders_products_nearfinal.pkl'))

In [74]:
ords_prods_merge.to_csv(os.path.join(path, 'Prepared Data', 'orders_products_nearfinal.csv'))

## After Task Activities:

### Attempt to find what costs 99999.0

In [78]:
# Checking for products taht have the 99999.0 prices
ords_prods_merge[ords_prods_merge['prices']==99999.0]

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_days,frequency_flag
29165516,183964,873,3,0,10,7.0,33664,11,0,2 % Reduced Fat Milk,...,99999.0,High-range product,Busiest day,Most orders,8,New customer,1523.47,High spender,10.0,Frequent customer
29165517,1851256,873,4,6,12,13.0,33664,8,1,2 % Reduced Fat Milk,...,99999.0,High-range product,Regularly busy,Most orders,8,New customer,1523.47,High spender,10.0,Frequent customer
29165518,1915696,1893,1,5,17,NaN,33664,10,0,2 % Reduced Fat Milk,...,99999.0,High-range product,Regularly busy,Average orders,6,New customer,4924.39,High spender,13.0,Regular customer
29165519,2763293,1893,2,4,16,13.0,33664,6,1,2 % Reduced Fat Milk,...,99999.0,High-range product,Least busy,Most orders,6,New customer,4924.39,High spender,13.0,Regular customer
29165520,2564805,1893,4,1,17,30.0,33664,3,1,2 % Reduced Fat Milk,...,99999.0,High-range product,Busiest day,Average orders,6,New customer,4924.39,High spender,13.0,Regular customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29166209,2249946,204099,29,0,8,4.0,33664,1,0,2 % Reduced Fat Milk,...,99999.0,High-range product,Busiest day,Average orders,39,Regular customer,1106.74,High spender,4.0,Frequent customer
29166210,2363282,204099,31,0,9,2.0,33664,1,1,2 % Reduced Fat Milk,...,99999.0,High-range product,Busiest day,Most orders,39,Regular customer,1106.74,High spender,4.0,Frequent customer
29166211,3181945,204395,13,3,15,8.0,33664,25,0,2 % Reduced Fat Milk,...,99999.0,High-range product,Least busy,Most orders,15,Regular customer,451.15,High spender,5.0,Frequent customer
29166212,2486215,205227,7,3,20,4.0,33664,8,0,2 % Reduced Fat Milk,...,99999.0,High-range product,Least busy,Average orders,12,Regular customer,1178.38,High spender,12.0,Regular customer


It looks to possibly be the same items, product_id 33664, 2 % Reduced Fat Milk.  If that is the case, I can just adjust the prices for that.  Of note, this is product id 33664... I swear there was a department in there before? Maybe it was a different sheet? Either way, I'll have to find a way to compare prices.

In [101]:
tempt_table = ords_prods_merge[ords_prods_merge['product_id']==33664]

KeyError: 'product_id'

In [83]:
tempt_table.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_days,frequency_flag
29165516,183964,873,3,0,10,7.0,33664,11,0,2 % Reduced Fat Milk,...,99999.0,High-range product,Busiest day,Most orders,8,New customer,1523.47,High spender,10.0,Frequent customer
29165517,1851256,873,4,6,12,13.0,33664,8,1,2 % Reduced Fat Milk,...,99999.0,High-range product,Regularly busy,Most orders,8,New customer,1523.47,High spender,10.0,Frequent customer
29165518,1915696,1893,1,5,17,NaN,33664,10,0,2 % Reduced Fat Milk,...,99999.0,High-range product,Regularly busy,Average orders,6,New customer,4924.39,High spender,13.0,Regular customer
29165519,2763293,1893,2,4,16,13.0,33664,6,1,2 % Reduced Fat Milk,...,99999.0,High-range product,Least busy,Most orders,6,New customer,4924.39,High spender,13.0,Regular customer
29165520,2564805,1893,4,1,17,30.0,33664,3,1,2 % Reduced Fat Milk,...,99999.0,High-range product,Busiest day,Average orders,6,New customer,4924.39,High spender,13.0,Regular customer


### Verifying if there is anything that costs that:

In [84]:
tempt_table.shape

(698, 22)

In [85]:
tempt_table['product_name'].value_counts()

product_name
2 % Reduced Fat  Milk    698
Name: count, dtype: int64

In [87]:
tempt_table['prices'].value_counts()

prices
99999.0    698
Name: count, dtype: int64

Okay, so using a temp. table, I found 698 entries for the 33664 product_id and all those entries are specifically for 2 % Reduced Fat Milk.  Additionally, all the prices are also 99999.0

In [88]:
# Realized I should also count how many product ID's have that price
temp_table = ords_prods_merge[ords_prods_merge['prices']==99999.0]

In [89]:
temp_table.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_days,frequency_flag
29165516,183964,873,3,0,10,7.0,33664,11,0,2 % Reduced Fat Milk,...,99999.0,High-range product,Busiest day,Most orders,8,New customer,1523.47,High spender,10.0,Frequent customer
29165517,1851256,873,4,6,12,13.0,33664,8,1,2 % Reduced Fat Milk,...,99999.0,High-range product,Regularly busy,Most orders,8,New customer,1523.47,High spender,10.0,Frequent customer
29165518,1915696,1893,1,5,17,NaN,33664,10,0,2 % Reduced Fat Milk,...,99999.0,High-range product,Regularly busy,Average orders,6,New customer,4924.39,High spender,13.0,Regular customer
29165519,2763293,1893,2,4,16,13.0,33664,6,1,2 % Reduced Fat Milk,...,99999.0,High-range product,Least busy,Most orders,6,New customer,4924.39,High spender,13.0,Regular customer
29165520,2564805,1893,4,1,17,30.0,33664,3,1,2 % Reduced Fat Milk,...,99999.0,High-range product,Busiest day,Average orders,6,New customer,4924.39,High spender,13.0,Regular customer


In [90]:
temp_table.shape

(698, 22)

In [94]:
temp_table['product_name'].value_counts()

product_name
2 % Reduced Fat  Milk    698
Name: count, dtype: int64

Okay now it is verified, issue here is... I have to find a way to compare milk prices.

Edit: Honestly, looking at the prices of other items, I'm almost certain the milk price should be $9.99 which is still a rip off but it is instacart...

In [97]:
# Went to chatGPT to find a way to find all entries with a certain word
milk_table = ords_prods_merge[ords_prods_merge['product_name'].str.contains('Milk')]

In [99]:
milk_table['prices'].value_counts()

prices
12.8    151867
1.3      67601
9.2      60618
7.9      55321
13.9     48822
         ...  
14.4       283
5.5        144
5.0        130
10.2       107
13.3        99
Name: count, Length: 143, dtype: int64

Well, looking at the prices, 9.99 is not out of the ordinary, okay, for sure we can just replace 99999.0 with 9.99

In [107]:
price_update_table = ords_prods_merge

In [110]:
price_update_table.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_days,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,Mid-range product,Least busy,Most orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,Mid-range product,Least busy,Most orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer


In [109]:
# Lesson learned, making a temp table LOL
price_update_table['prices'] = ords_prods_merge['prices'].replace({99999.0:9.99})

In [115]:
check_tempt = price_update_table[price_update_table['product_id']==33664]

In [116]:
check_tempt['prices'].value_counts()

prices
9.99    698
Name: count, dtype: int64

In [120]:
price_update_table.shape

(32404859, 22)

In [123]:
# Everything looks good, can export it but I saw someone else's work...
# and they had an amazing line that checked max prices and listed them:
price_update_table['prices'].drop_duplicates().sort_values(ascending = False)

10030345    14900.0
18504754       25.0
25758883       24.9
28425432       24.8
14484538       24.7
             ...   
3384024         1.4
632766          1.3
5488887         1.2
5147325         1.1
60725           1.0
Name: prices, Length: 242, dtype: float64

basically there is an item in there, priced 14900.0, time to do everything all over again.

### Round Two, for 14900.0:

In [125]:
# Checking for products taht have the 99999.0 prices
price_update_table[price_update_table['prices']==14900.0]

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_days,frequency_flag
10030345,912404,17,12,2,14,5.0,21553,5,0,Lowfat 2% Milkfat Cottage Cheese,...,14900.0,High-range product,Regularly busy,Most orders,40,Regular customer,108.65,High spender,5.0,Frequent customer
10030346,603376,17,22,6,16,4.0,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,...,14900.0,High-range product,Regularly busy,Most orders,40,Regular customer,108.65,High spender,5.0,Frequent customer
10030347,3264360,135,2,2,21,13.0,21553,6,0,Lowfat 2% Milkfat Cottage Cheese,...,14900.0,High-range product,Regularly busy,Average orders,4,New customer,1154.79,High spender,12.0,Regular customer
10030348,892534,135,3,0,8,12.0,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,...,14900.0,High-range product,Busiest day,Average orders,4,New customer,1154.79,High spender,12.0,Regular customer
10030349,229704,342,8,1,19,30.0,21553,9,0,Lowfat 2% Milkfat Cottage Cheese,...,14900.0,High-range product,Busiest day,Average orders,16,Regular customer,114.43,High spender,23.0,Non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10034769,3172853,205650,18,1,9,7.0,21553,17,1,Lowfat 2% Milkfat Cottage Cheese,...,14900.0,High-range product,Busiest day,Most orders,25,Regular customer,351.14,High spender,6.0,Frequent customer
10034770,2504315,205818,3,5,15,3.0,21553,13,0,Lowfat 2% Milkfat Cottage Cheese,...,14900.0,High-range product,Regularly busy,Most orders,25,Regular customer,170.07,High spender,13.0,Regular customer
10034771,1108388,205818,5,4,5,1.0,21553,5,1,Lowfat 2% Milkfat Cottage Cheese,...,14900.0,High-range product,Least busy,Fewest orders,25,Regular customer,170.07,High spender,13.0,Regular customer
10034772,1916142,206049,1,2,17,NaN,21553,2,0,Lowfat 2% Milkfat Cottage Cheese,...,14900.0,High-range product,Regularly busy,Average orders,5,New customer,938.03,High spender,7.0,Frequent customer


In [127]:
tmp1 = price_update_table[price_update_table['prices']==14900.0]

In [130]:
tmp1['product_id'].value_counts()

product_id
21553    4429
Name: count, dtype: int64

In [131]:
tmp2 = price_update_table[price_update_table['product_name'].str.contains('Cottage Cheese')]

In [133]:
tmp2['prices'].value_counts()

prices
8.2        11570
8.4         9225
9.4         7900
2.0         4606
14900.0     4429
13.7        3767
5.6         3394
6.9         2910
6.7         2480
8.0         2473
7.7         2397
11.4        2153
11.5        1633
3.0         1456
3.9         1168
10.5        1030
1.9          839
6.4          706
1.3          702
9.2          663
5.8          639
3.4          551
8.6          551
4.7          546
2.1          505
1.1          476
9.7          442
9.3          433
2.2          374
2.4          350
2.7          322
11.9         261
3.7          250
1.8          246
6.5          235
1.6          219
7.0          169
3.3          155
2.3          142
4.9          116
12.0         114
6.1           90
15.0          87
7.5           79
7.2           74
5.3           69
13.5          63
6.8           49
11.6          45
3.8           31
8.9           28
9.5           20
12.1          15
3.1           15
11.8          14
12.6          12
12.3           6
10.4           5
9.6    

In [134]:
price_update_table['prices'] = price_update_table['prices'].replace({14900.0:14.90})

In [135]:
price_update_table['prices'].drop_duplicates().sort_values(ascending = False)

18504754    25.0
25758883    24.9
28425432    24.8
14484538    24.7
23374297    24.6
            ... 
3384024      1.4
632766       1.3
5488887      1.2
5147325      1.1
60725        1.0
Name: prices, Length: 241, dtype: float64

In [138]:
price_update_table.to_pickle(os.path.join(path,'Prepared Data', 'orders_products_nearfinal.pkl'))

In [139]:
price_update_table.to_csv(os.path.join(path,'Prepared Data','orders_products_nearfinal.csv'))